In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

# Load and preprocess the dataset
celeb_data = pd.read_csv('../list_attr_celeba.csv')
celeb_data.replace(-1, 0, inplace=True)
celeb_data.drop(columns=[
    'image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Bags_Under_Eyes',
    'Bushy_Eyebrows', 'Bangs', 'Blurry', 'Smiling', 'Mouth_Slightly_Open',
    'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
    'Wearing_Necktie'
], inplace=True)

# Define features (X) and target (y)
X = celeb_data.drop("Attractive", axis=1)
y = celeb_data["Attractive"]

# Feature Selection (L1 Regularization)
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(penalty='l1', solver='liblinear', random_state=42, C=1.0)
logistic.fit(X, y)
coefficients = logistic.coef_[0]
selected_features = X.columns[coefficients != 0]
X_selected = X[selected_features]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)  # Add an extra dimension for binary classification
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# Define the Feedforward Neural Network
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)  # First hidden layer with 32 neurons
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 16)         # Second hidden layer with 16 neurons
        self.fc3 = nn.Linear(16, 1)          # Output layer
        self.sigmoid = nn.Sigmoid()          # Sigmoid for binary classification
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize the model, loss function, and optimizer
input_size = X_train_tensor.shape[1]
model = NeuralNetwork(input_size)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred_probs = model(X_test_tensor)
    y_pred = (y_pred_probs >= 0.5).float()  # Convert probabilities to binary predictions

# Convert predictions to numpy for sklearn evaluation
y_pred_numpy = y_pred.squeeze().numpy()
y_test_numpy = y_test_tensor.squeeze().numpy()

# Print evaluation metrics
print("\nClassification Report:")
print(classification_report(y_test_numpy, y_pred_numpy))

Epoch [10/50], Loss: 0.6883
Epoch [20/50], Loss: 0.6803
Epoch [30/50], Loss: 0.6677
Epoch [40/50], Loss: 0.6490
Epoch [50/50], Loss: 0.6228

Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.49      0.61     29734
         1.0       0.65      0.90      0.75     31046

    accuracy                           0.70     60780
   macro avg       0.74      0.70      0.68     60780
weighted avg       0.74      0.70      0.69     60780



In [4]:
# accuracy
accuracy = accuracy_score(y_test_numpy, y_pred_numpy)
print(f"Accuracy: {accuracy:.6f}")

# precision
precision = precision_score(y_test_numpy, y_pred_numpy)
print(f"Precision: {precision:.6f}")

# recall
recall = recall_score(y_test_numpy, y_pred_numpy)
print(f"Recall: {recall:.6f}")

# F1 score
f1 = f1_score(y_test_numpy, y_pred_numpy)
print(f"F1 Score: {f1:.6f}")

Accuracy: 0.699654
Precision: 0.647904
Recall: 0.902403
F1 Score: 0.754264
